In [1]:
import random
import math
import numpy as np
import Parsing_midi as pm
import Init_chromo as init
import Melody_matching as mm
import count_CE as ce
import mido
from mido import Message, MidiFile, MidiTrack
from collections import Counter
import copy
import matplotlib.pyplot as plt
from itertools import combinations # product: 排列
import time
import multiprocessing as mp
import pretty_midi

/home/hsuan/env1/lib/python3.6/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [20]:
midi_list = pm.get_midi('MidiSample/ori.mid', 0)

def trans_note_list(song):
    return [song[i].note for i in range(len(song))]

note_msg, midi_msg = pm.separate_song(midi_list, 9600)
parent = midi_msg[0]  # can change here for testing

def flatten_list(midi_list):
    return [item for sublist in midi_list for item in sublist]

## 4 beats (1920) in one bar
note_msg, midi_msg = pm.separate_song(parent, 1920)
origin_note = [trans_note_list(midi_msg[i]) for i in range(len(midi_msg))]

CE = ce.get_ce_list(midi_msg)  # CE list from every bar

note = trans_note_list(midi_msg[2])

for i in range(len(midi_msg)):
    print(fitness(midi_msg[i], CE[i], origin_note[i]))

(0.0, [<message note_on channel=0 note=71 velocity=113 time=1>, <message note_on channel=0 note=71 velocity=0 time=1439>, <message note_on channel=0 note=69 velocity=113 time=1>, <message note_on channel=0 note=69 velocity=0 time=239>, <message note_on channel=0 note=71 velocity=113 time=1>, <message note_on channel=0 note=71 velocity=0 time=239>])
(0.0, [<message note_on channel=0 note=72 velocity=113 time=1>, <message note_on channel=0 note=72 velocity=0 time=719>, <message note_on channel=0 note=71 velocity=113 time=1>, <message note_on channel=0 note=71 velocity=0 time=239>, <message note_on channel=0 note=72 velocity=113 time=1>, <message note_on channel=0 note=72 velocity=0 time=479>, <message note_on channel=0 note=76 velocity=113 time=1>, <message note_on channel=0 note=76 velocity=0 time=479>])
(-0.25, [<message note_on channel=0 note=71 velocity=113 time=1>, <message note_on channel=0 note=71 velocity=0 time=1439>, <message note_on channel=0 note=64 velocity=113 time=1>, <mes

In [18]:
C_pitch = [0, -1, 2, -1, 4, 5, -1, 7, -1, 9, -1, 11]  # there is no sharp and flat in C scale, so set it as -1
chord_list = [[0,4,7],[-1,-1,-1],[2,5,9],[-1,-1,-1],[4,7,11],[5,9,0],[-1,-1,-1],[7,11,2],[-1,-1,-1],[9,0,4],[-1,-1,-1],[11,2,5]]
pitch_list = ['C','D','E','F','G','A','B']

harmony = [[0,2,4,7,-3,-5,-1], [0], [0,5,-2,-3,-7,2,3],[0], [0,3,1,-4,-2,-7], [0,2,4,-1,-3,-6],\
[0], [0,2,4,7,-2,-3,-5], [0],[0,2,3,7,-2,-7,-5],[0],[1,3,0,-4,-2,-9]]

invalid = [[],[0],[],[0],[],[],[0],[],[0],[],[0],[]]


def fitness(midi_msg, chord, ori_note): # input type: list
    score_ce = 0; score_harmony = 0
    scale_list = [0,2,4,5,7,9,11]  # = midi notes = ['C','D','E','F','G','A','B']
    
#     # rate the CE distance in each bar (30%)
#     xyz_ = ce.trans_to_coordinate(midi_msg)
#     CE_ = ce.count_CE(xyz_)  
#     dist = ce.count_distance(chord, CE_)  # compare new and old CE
#     if dist == 0.0:  # the chord remain
#         score_ce = 40
#     elif dist > 0 and dist <= 2:
#         score_ce = -5*(dist)
#     elif dist > 3 and dist <= 4:
#         score_ce = -7*(dist)
#     elif dist > 4 and dist <= 8:
#         score_ce = -8*(dist)
#     else:
#         score_ce = -30     
    
    ## get chord note, ie: C major is CEG
    c = int(list(chord)[2])
    if c % 12 in C_pitch:
        chord = chord_list[c % 12]
    elif c + 1 % 12 in C_pitch:
        chord = chord_list[c + 1 % 12]
    elif c - 1 % 12 in C_pitch:
        chord = chord_list[c - 1 % 12]
    
    ## score every notes based on harmony rule (40%)
    for i in range(len(midi_msg)):  # TODO: if len==1
        if i % 2 == 0:
            if midi_msg[i].note in ori_note: # keep original melody
                score_harmony += 3
            elif midi_msg[i].note not in ori_note:
                score_harmony -= 2                         
            if midi_msg[i].note % 12 == chord[0]:  # if the note is chord root note
                score_harmony += 6
            elif midi_msg[i].note % 12 == chord[1]:  # if the note is 2nd note
                score_harmony += 4
            elif midi_msg[i].note % 12 == chord[2]:  # if the note is 3rd note
                score_harmony += 4            
            if i == 0 and midi_msg[i].note % 12 == chord[0]:  # the note is the first note and is a chord root note
                score_harmony += 6
            elif i == 0 and midi_msg[i].note % 12 == chord[1]:  # the note is the first note and is a 2nd chord note
                score_harmony += 4
            elif i == 0 and midi_msg[i].note % 12 == chord[2]:  # the note is the first note and is a 3rd chord note
                score_harmony += 4 
            if i == len(midi_msg)-2 and midi_msg[i].note % 12 == chord[0]: # the note is the last note and is a chord root note
                score_harmony += 6
            elif i == len(midi_msg)-2 and midi_msg[i].note % 12 == chord[1]: # the note is the last note and is a 2nd chord note
                score_harmony += 4
            elif i == len(midi_msg)-2 and midi_msg[i].note % 12 == chord[2]: # the note is the last note and is a 3rd chord note
                score_harmony += 4
            
            if i == 0 and midi_msg[i].note % 12 not in chord: # not in chord note
                score_harmony -= 5                   
            if i == len(midi_msg)-2 and midi_msg[i].note % 12 not in chord: # not in chord note
                score_harmony -= 8
            if midi_msg[i].note % 12 not in chord: # note is not a chord note
                score_harmony -= 8
            if midi_msg[i].note % 12 not in scale_list:  # note not in the scale (C major)
                score_harmony -= 20            
            if i < len(midi_msg) - 3 and abs(midi_msg[i].note - midi_msg[i+2].note) >= 7:  # big jump notes
                score_harmony -= 30
            if i < len(midi_msg) - 3 and abs(midi_msg[i].note - midi_msg[i+2].note) == 0:  # repetitive notes
                score_harmony -= 10            
                      
            ## 每個音和諧的degree
            harmony_list = harmony[midi_msg[i].note % 12]
            if i < len(midi_msg) - 3 and midi_msg[i+2].note - midi_msg[i].note in harmony_list:
                score_harmony += 4
            if i < len(midi_msg) - 3 and midi_msg[i+2].note - midi_msg[i].note not in harmony_list:
                score_harmony -= 10
              
            # duration
            if midi_msg[i].time - midi_msg[i+1].time > 65 and midi_msg[i].time - midi_msg[i+1].time < 125: 
                score_harmony += 6
            if i < len(midi_msg) - 3 and midi_msg[i+2].time - midi_msg[i].time > 200:
                score_harmony -= 4    
            if midi_msg[i].time - midi_msg[i+1].time > 475: 
                score_harmony -= 10
                
            elif midi_msg[i].time - midi_msg[i+1].time < 65: # the duration of the note is too short
                score_harmony -= 8       
        score_harmony /= (len(midi_msg))   
        
    return int(score_ce + score_harmony)/4 , midi_msg  # 4 bar